# TODO

1. Make graphs more interactive

In [2]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
import geopandas as gpd
import pandas as pd
import json

In [3]:
defaultDate = date(2021, 4, 29)
print(type(str(defaultDate)))

<class 'str'>


In [4]:
# import naturalearth_lowres (country coordinates)
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world = world[['name', 'iso_a3', 'geometry']]
world.columns = ['country', 'Alpha-3 code', 'geometry']

In [5]:
# convert iso3 to iso2 codes
iso = pd.read_csv('./data/iso-country-codes.csv')
iso = iso[['Alpha-3 code', 'Alpha-2 code']]
world = world.merge(iso, on='Alpha-3 code', how='left')
world = world[['country', 'Alpha-2 code', 'geometry']]
world.columns = ['country', 'country_code', 'geometry']

In [6]:
# import air pollution data
temp = pd.read_csv('./data/avgOfMedians.csv', names=['country_code', 'date', 'type', 'avg'])
data = temp[4:]

/Users/hyunsoo_kim/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# create different json for entire data
# o3
o3 = data[data['type'] == 'o3']
o3_merged = world.merge(o3, on='country_code', how='left')
o3_merged['type'].fillna('o3', inplace=True)
o3_df = json.loads(o3_merged.to_json())
o3_data = json.dumps(o3_df)

# pm10
pm10 = data[data['type'] == 'pm10']
pm10_merged = world.merge(pm10, on='country_code', how='left')
pm10_merged['type'].fillna('pm10', inplace=True)
pm10_df = json.loads(pm10_merged.to_json())
pm10_data = json.dumps(pm10_df)

# pm25
pm25 = data[data['type'] == 'pm25']
pm25_merged = world.merge(pm25, on='country_code', how='left')
pm25_merged['type'].fillna('pm25', inplace=True)
pm25_df = json.loads(pm25_merged.to_json())
pm25_data = json.dumps(pm25_df)

# co
co = data[data['type'] == 'co']
co_merged = world.merge(co, on='country_code', how='left')
co_merged['type'].fillna('co', inplace=True)
co_df = json.loads(co_merged.to_json())
co_data = json.dumps(co_df)

# no2
no2 = data[data['type'] == 'no2']
no2_merged = world.merge(no2, on='country_code', how='left')
no2_merged['type'].fillna('no2', inplace=True)
no2_df = json.loads(no2_merged.to_json())
no2_data = json.dumps(no2_df)

# so2
so2 = data[data['type'] == 'so2']
so2_merged = world.merge(so2, on='country_code', how='left')
so2_merged['type'].fillna('so2', inplace=True)
so2_df = json.loads(so2_merged.to_json())
so2_data = json.dumps(so2_df)

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"avg": null'

In [19]:
# retrieve 2021-01-01 (default) data
o3_2021_04_29_merged = o3_merged[o3_merged['date'] == '2021-04-29']
o3_2021_04_29_df = json.loads(o3_2021_04_29_merged.to_json())
o3_2021_04_29_data = json.dumps(o3_2021_04_29_df)

so2_2021_04_29_merged = so2_merged[so2_merged['date'] == '2021-04-29']
so2_2021_04_29_df = json.loads(so2_2021_04_29_merged.to_json())
so2_2021_04_29_data = json.dumps(so2_2021_04_29_df)

In [16]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.io import curdoc, output_notebook
from bokeh.models import DateSlider, HoverTool, Panel, Slider, Dropdown
from bokeh.layouts import widgetbox, row, column
from bokeh.models.callbacks import CustomJS

In [11]:
# for o3
geosource = GeoJSONDataSource(geojson = o3_data)
# geosource = GeoJSONDataSource(geojson = o3_2021_01_01_data)

In [12]:
displayed_source = GeoJSONDataSource(geojson = o3_2021_04_29_data)

In [13]:
tick_labels = {}
for i in range(0, 40):
    s = str(i)
    tick_labels[s] = s

so2_merged.head()

,country,country_code,geometry,date,type,avg
0,Fiji,FJ,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",NaN,so2,NaN
1,Tanzania,TZ,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",NaN,so2,NaN
2,W. Sahara,EH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",NaN,so2,NaN
3,Canada,CA,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",2020-12-28,so2,0.8899999999999999
4,Canada,CA,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",2020-12-29,so2,0.8599999999999998


In [14]:
# palette = brewer['YlGnBu']
# palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
low = 0 # TODO
high = 40 # TODO
color_mapper = LinearColorMapper(palette = "Viridis256", low = low, high = high, nan_color='#d9d9d9')

# Initialize figure object.
p = figure(title = 'O3 concentration on 2021-04-29', plot_height = 600 , plot_width = 950)
# p.xgrid.grid_line_color = None
# p.ygrid.grid_line_color = None

# Add patch renderer for filling in country 
p.patches('xs','ys', source = displayed_source, fill_color = {'field' :'avg', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

# Add color bar
# tick_labels = {'30':'30','31':'31', '32':'32', '33':'33', '34':'34', '35':'35', '36':'36', '37':'37', '38':'38', '39':'39',  '40': '>40'} 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color='grey',location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
p.add_layout(color_bar, 'below')

#Display figure
# show(p)

In [23]:
start = 0
end = 200
from datetime import date
slider = DateSlider(title = 'date', start = date(2021, 1, 1), end = date(2021, 10, 1), step = 1, value = date(2021, 4, 29))

slider_callback = CustomJS(args=dict(slider=slider, source=geosource, displayed_src=displayed_source), code="""
    var s_value = slider.value 
    var date = (new Date(s_value)).toISOString().slice(0, 10)
       
    var columns = Object.keys(source.data)
    var data = {}
    columns.forEach(function(key) {
        data[key] = [];
    });
    for( var i = 0; i < source.get_length(); i++){
        if(source.data['date'][i] == date || !source.data['date'][i]){
            columns.forEach((key) => {
                data[key].push(source.data[key][i])
            })
        }
    }
    displayed_src.data = data;
    console.log(displayed_src.data)
    displayed_src.change.emit();
""")

slider.js_on_change('value', slider_callback)


# Update figure title from slider change
title_callback = CustomJS(args=dict(slider=slider, figure=p), code="""
    var num = slider.value;
    var date = (new Date(num)).toISOString().slice(0, 10)
    figure.title.text = 'O3 concentration on ' + date;
""")
slider.js_on_change('value', title_callback)

air_indices = [("O3", "o3"), ("SO2", "so2"), ("CO", "co"), ("PM10", "pm10"), ("PM25", "pm25")]
dropdown = Dropdown(label="Air pollution index", button_type="default", menu=air_indices)
dropdown_callback = CustomJS(args=dict(source=geosource, displayed_src=displayed_source), code="""
    var index = this.item
    console.log(index)
    var data = {}
//    if (index == "so2")
        data = so2_data
    source = GeoJSONDataSource(geojson = data)
    displayed_src = GeoJSONDataSource(geojson=so2_2021_04_29_data)
    displayed_src.change.emit();
    source.change.emit();
    
""")
dropdown.js_on_event("menu_item_click", dropdown_callback)

layout = column(p, widgetbox(slider, dropdown))

curdoc().clear()
curdoc().add_root(layout)

show(layout)

# ----------------------------

In [18]:


menu = [("Item 1", "item_1"), ("Item 2", "item_2"), None, ("Item 3", "item_3")]

dropdown = Dropdown(label="Dropdown button", button_type="warning", menu=menu)
dropdown.js_on_event("menu_item_click", CustomJS(code="console.log('dropdown: ' + this.item, this.toString())"))

show(dropdown)
